In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import accuracy_score
%matplotlib  inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/titanic/train.csv")
test=pd.read_csv("../input/titanic/test.csv")


In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
sns.heatmap(df.isnull(),yticklabels = False , cbar= False , cmap = 'viridis')


In [ ]:
corr = df.corr()
plt.figure(figsize=(20,15))
sns.heatmap(corr, annot = True)
plt.show()

In [ ]:
values = df['Survived'].value_counts()
labels = ['Not Survived', 'Survived']
fig, ax = plt.subplots(figsize = (5, 5), dpi = 100)
explode = (0, 0.06)
patches, texts, autotexts = ax.pie(values, labels = labels, autopct = '%1.2f%%', 
                                   startangle = 90, explode = explode)

plt.show()

In [ ]:
plt.figure(figsize = (16, 8))
sns.countplot('Pclass', hue = 'Survived', data = df)
plt.show()

In [ ]:
plt.figure(figsize = (16, 7))
sns.countplot('Sex', hue = 'Survived', data = df)
plt.show()

In [ ]:
plt.pie(df.groupby('Survived').count()['Sex'][:2],labels=df['Sex'].unique()[:2],explode=[0,.05,],autopct='%1.1f%%');

In [ ]:
plt.figure(figsize = (14, 6))

sns.countplot('Embarked', hue = 'Survived', data = df)
plt.show()

In [ ]:
plt.figure(figsize = (16, 5))

sns.countplot(x = 'SibSp', data = df, hue = 'Survived')
plt.show()

In [ ]:
df.drop([ 'Ticket', 'Cabin'], axis = 1, inplace = True)
test.drop([ 'Ticket', 'Cabin'], axis = 1, inplace = True)

In [ ]:
corr = df.corr()
col = corr['Survived'].sort_values(ascending = False).abs()
col

In [ ]:
total = df.isna().sum().sort_values(ascending=False)
missing_data = pd.concat([total], axis=1, keys=["Total"])
missing_data.head(30)

In [ ]:
df['Fare'] = df['Fare'].replace(0, df['Fare'].mean())
test['Fare'].fillna(test['Fare'].mean(), inplace = True)
test['Fare'] = test['Fare'].replace(0, test['Fare'].mean())


In [ ]:
#df['Age'].fillna(df['Age'].mean(), inplace = True)
#test['Age'].fillna(test['Age'].mean(), inplace = True)

In [ ]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace = True)

In [ ]:
df['Sex'] = df['Sex'].apply(lambda val: 1 if val == 'male' else 0)
test['Sex'] = test['Sex'].apply(lambda val: 1 if val == 'male' else 0)

In [ ]:
df['Embarked'] = df['Embarked'].map({'S' : 0, 'C': 1, 'Q': 2})
test['Embarked'] = test['Embarked'].map({'S' : 0, 'C': 1, 'Q': 2})

In [ ]:
total = df.isna().sum().sort_values(ascending=False)
missing_data = pd.concat([total], axis=1, keys=["Total"])
missing_data.head(30)

In [ ]:
sns.heatmap(df.isnull(),yticklabels = False , cbar= False , cmap = 'viridis')

In [ ]:
df['Initial']=0
for i in df:
    df['Initial']=df.Name.str.extract('([A-Za-z]+)\.')
test['Initial']=0
for i in test:
    test['Initial']=test.Name.str.extract('([A-Za-z]+)\.')
pd.crosstab(df.Initial,df.Sex).T.style.background_gradient() #Checking the Initials with the Sex
df['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)
test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Dona','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)
## Assigning the NaN Values with the Ceil values of the mean ages
df.loc[(df.Age.isnull())&(df.Initial=='Mr'),'Age']=33
df.loc[(df.Age.isnull())&(df.Initial=='Mrs'),'Age']=36
df.loc[(df.Age.isnull())&(df.Initial=='Master'),'Age']=5
df.loc[(df.Age.isnull())&(df.Initial=='Miss'),'Age']=22
df.loc[(df.Age.isnull())&(df.Initial=='Other'),'Age']=46

#doing the same for the test
## Assigning the NaN Values with the Ceil values of the mean ages
test.loc[(test.Age.isnull())&(test.Initial=='Mr'),'Age']=33
test.loc[(test.Age.isnull())&(test.Initial=='Mrs'),'Age']=36
test.loc[(test.Age.isnull())&(test.Initial=='Master'),'Age']=5
test.loc[(test.Age.isnull())&(test.Initial=='Miss'),'Age']=22
test.loc[(test.Age.isnull())&(test.Initial=='Other'),'Age']=46


In [ ]:
X=df[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
y=df[["Survived"]]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
ranfor = RandomForestClassifier()
ranfor.fit(X_train,y_train)
predictions = ranfor.predict(X_test)
accuracy_score(y_test, predictions)

In [ ]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
d_predictions = dtree.predict(X_test)
accuracy_score(y_test, d_predictions)

In [ ]:
log_model = LogisticRegression()
log_model.fit(X_train,y_train)
l_predictions = log_model.predict(X_test)
accuracy_score(y_test, l_predictions)

In [ ]:
Test=test[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
log_model = LogisticRegression()
log_model.fit(X_train,y_train)
l_predictions = log_model.predict(Test)




In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId,
                       'Survived': l_predictions})
output.to_csv('submission.csv', index=False)

In [ ]:
output